# XGBoots

In [28]:
# imports
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Read the Dataframe here
#load data
jan_2019 = pd.read_csv('../Kyle_eda/data/01_2019_flights_weather.csv', low_memory=False)
# drop rows with nan in arr_delay
jan_2019.dropna(inplace=True,subset=['arr_delay'])

# Read test Dataframe here
# test_data =

In [5]:
# encode mkt_unique_carrier
enc = OneHotEncoder()
enc.fit(jan_2019['mkt_unique_carrier'].values.reshape(-1,1))
enc_df = pd.DataFrame(enc.transform(jan_2019['mkt_unique_carrier'].values.reshape(-1,1)).toarray())

enc_df.reset_index(drop=True, inplace=True)
jan_2019.reset_index(drop=True, inplace=True)
jan_2019 = pd.concat([jan_2019, enc_df], axis=1,)

In [6]:
# get mapping dictionary from file to use for encoding origin and dest
dest_dict = pd.read_csv('../Kyle_eda/data/dest_dict.csv', index_col=0)
origin_dict = pd.read_csv('../Kyle_eda/data/origin_dict.csv', index_col=0)
dest_dict = dest_dict.T.to_dict('records')[0]
origin_dict = origin_dict.T.to_dict('records')[0]

In [7]:
# encode origin and dest
jan_2019['dest'] = jan_2019['dest'].map(dest_dict)
jan_2019['origin'] = jan_2019['origin'].map(origin_dict)


,date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,0,1,2,3,4,5,6,7,8,9
0,2019-01-01,AA,AA_CODESHARE,AA,5621,OH,N720PS,5621,11057,7.062739,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-01,AA,AA,AA,1950,AA,N831AW,1950,11057,7.062739,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-01,DL,DL,DL,354,DL,N976DL,354,10397,2.697050,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-01,DL,DL,DL,898,DL,N319US,898,12953,6.358496,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-01,B6,B6,B6,2953,B6,N949JB,2953,12478,2.662853,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617644,2019-01-31,HA,HA,HA,116,HA,N477HA,116,12173,0.482318,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
617645,2019-01-31,HA,HA,HA,122,HA,N493HA,122,12173,0.482318,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
617646,2019-01-31,HA,HA,HA,123,HA,N481HA,123,12173,0.482318,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
617647,2019-01-31,HA,HA,HA,138,HA,N479HA,138,12173,0.482318,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
# mask for columns to keep
cols_to_keep = ['origin', 
                'dest', 
                'crs_elapsed_time',
                'distance',
                'crs_dep_time_hr',
                'crs_arr_time_hr',
                'precip_origin',
                'windspeedKmph_origin',
                'winddirDegree_origin',
                'visibility_origin',
                'DewPointC_origin',
                'pressure_origin',
                'cloudcover_origin',
                'WindGustKmph_origin',
                'humidity_origin',
                'tempC_origin',
                'precip_dest',
                'windspeedKmph_dest',
                'winddirDegree_dest',
                'visibility_dest',
                'DewPointC_dest',
                'pressure_dest',
                'cloudcover_dest',
                'WindGustKmph_dest',
                'humidity_dest',
                'tempC_dest']

X = jan_2019[cols_to_keep]
y = jan_2019['arr_delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# Standard Scaler creation using the fit_transform() method
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [11]:
#Create the model
xgboost =  xgb.XGBRegressor(objective = 'reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                          max_depth = 5, alpha = 10, n_estimators = 10)

#Train the model 
xgboost.fit(scaled_X_train,y_train)

[23:20:07] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=6, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [12]:
# Preds
y_pred = xgboost.predict(scaled_X_test)

In [24]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 51.7066044900115


In [ ]:
# Confusion Matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Classification Report
print(classification_report(y_test, y_pred))

In [ ]:
# Create a Dataframe from Pred
pred = pd.DataFrame(y_pred)

### Hypertuning 

In [33]:
# Create function for Hypertuning

def hypertuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100,300,500],
        'objective': ['reg:squarederror']}

    xgb_model = xgb.XGBRegressor()

    grid_search = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    grid_search.fit(X_train,y_train)

    return grid_search.best_params_


# Calling function
hypertuning(scaled_X_train,y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


KeyboardInterrupt: 

In [ ]:
# Put the best stimated here
xgb_model = XGBRegressor(
        objective = ,
        colsample_bytree = ,
        learning_rate = ,
        max_depth = ,
        min_child_weight =,
        n_estimators = ,
        alpha = )

# fit the model
xgb_model.fit(X_train, y_train)

# Predictions
training_preds = xgb_model.predict(X_train)
y_pred = xgb_model.predict(X_test)


In [ ]:
# Classification Report
print(classification_report(y_test, y_pred))

In [ ]:
# RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# R2 score
r2 = r2_score(y_test,y_pred)

print("RMSE: %.4f" % (rmse))
print('r2_score:', r2)